In [1]:
import numpy as np
import torch
import json
import os

from libraries.dataset        import standardize_dataset
from libraries.graph          import graph_POSCAR_encoding
from libraries.structure      import compute_diffraction_pattern
from torch_geometric.data     import Data
from pymatgen.core            import Structure, Molecule, Lattice

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/claudio/cibran/Work/UPC/GenerativeModels/venv/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# In case database is created from scratch (otherwise, it is not being used)
data_path = '/home/claudio/cibran/Work/UPC/CLUE/Loaded_PhaseTransition'

# Type of data in folder
data_type = 'POSCAR'  # 'POSCAR' or 'xyz'

# Define diffraction type (None, 'neutron', 'xrd', 'EPA' or other)
targets = ['H', 'Cv', 'gap']

# Define folder in which all data will be stored
data_folder = f'QM9-all-linked'

encoding_type      = 'all-linked'  # 'voronoi', 'all-linked' or 'sphere-images'
distance_threshold = None  # 6, used in general
periodicity        = None  # Better False for molecules, always True for crystals

# Define basic dataset parameters for tracking data
dataset_parameters = {
    'input_folder': data_path,
    'output_folder': data_folder,
    'target': targets,
    'encoding_type': encoding_type,
    'distance_threshold': distance_threshold
}

if not os.path.exists(data_folder):
    os.system(f'mkdir {data_folder}')

# Dump the dictionary with numpy arrays to a JSON file
with open(f'{data_folder}/dataset_parameters.json', 'w') as json_file:
    json.dump(dataset_parameters, json_file)

# Generation of graph database for training

Load the datasets, already standardized if possible.

In [ ]:
# Generate the raw dataset from scratch, and standardize it

# Read all materials within the database
materials = os.listdir(data_path)

dataset = []
for material in materials:
    try:
        # Try to read the polymorphs
        polymorphs = os.listdir(f'{data_path}/{material}')
    except:
        continue
    
    print(material)
    for polymorf in polymorphs:
        # Path to folder containing the POSCAR
        path_to_POSCAR = f'{data_path}/{material}/{polymorf}'
        
        # Check that the folder is valid
        if os.path.exists(path_to_POSCAR):
            print(f'\t{polymorf}')
            
            try:
                if data_type == 'POSCAR':
                    # Load pymatgen structure object
                    structure = Structure.from_file(f'{path_to_POSCAR}/POSCAR')
                elif data_type == 'xyz':
                    molecule  = Molecule.from_file(path_to_POSCAR)
                    
                    # Define the molecule within a [100, 100, 100] POSCAR lattice
                    structure = Structure(Lattice.cubic(100), molecule.species, molecule.cart_coords, coords_are_cartesian=True)
                    
                    with open(path_to_POSCAR, 'r') as file:
                        lines = file.readlines()
                    properties_values = lines[1].split()
                    
                    properties_tags = ['tag', 'index', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']
                    
                    graph_level_data = {properties_tags[i]: properties_values[i] for i in enumerate(properties_tags)}
                
                nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                                 encoding_type=encoding_type,
                                                                 distance_threshold=distance_threshold,
                                                                 periodicity=periodicity)
            except:
                print(f'Error: {material} {polymorf} not loaded')
                continue

            extracted_target = []
            for target in targets:
                if target == 'EPA':
                    # Load ground state energy per atom
                    extracted_target.append(float(np.loadtxt(f'{path_to_POSCAR}/EPA')))
                elif target == 'bandgap':
                    # Load band-gap
                    extracted_target.append(float(np.loadtxt(f'{path_to_POSCAR}/bandgap')))
                elif (target == 'neutron') or (target == 'xrd'):
                    # Compute diffraction pattern from given structure
                    extracted_target.append(compute_diffraction_pattern(structure, diffraction=target))
                elif None:
                    # Do not extract anything
                    extracted_target.append(0)
                else:
                    extracted_target.append(float(graph_level_data[target]))
            
            # Construct temporal graph structure
            graph = Data(x=nodes,
                         edge_index=edges.t().contiguous(),
                         edge_attr=attributes.ravel(),
                         y=torch.tensor(extracted_target, dtype=torch.float),
                         label=f'{material} {polymorf}'
                        )

            # Append to dataset and labels
            dataset.append(graph)

In [ ]:
# Standardize dataset
dataset_std, labels_std, dataset_parameters = standardize_dataset(dataset, transformation='inverse-quadratic')

# Save dataset

In [ ]:
dataset_name                = f'{data_folder}/dataset.pt'
dataset_name_std            = f'{data_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{data_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

torch.save(dataset,     dataset_name)
torch.save(dataset_std, dataset_name_std)

# Convert torch tensors to numpy arrays
numpy_dict = {}
for key, value in dataset_parameters.items():
    try:
        numpy_dict[key] = value.cpu().numpy().tolist()
    except:
        numpy_dict[key] = value

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name_std, 'w') as json_file:
    json.dump(numpy_dict, json_file)